In [0]:
%sql
create schema datamodel.gold;

#### dim_customers

In [0]:
%sql
create or replace table datamodel.gold.dim_customers
as
with dedup (
    select distinct
        customer_id,
        customer_name,
        customer_email,
        customer_email_host
    from datamodel.silver.silver_tbl
)
select 
    row_number() over (order by customer_id) as dim_customer_key,
    *
from dedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodel.gold.dim_customers;

dim_customer_key,customer_id,customer_name,customer_email,customer_email_host
1,1,Alice Johnson,alice@gmail.com,gmail.com
2,2,Bob Smith,bob@yahoo.com,yahoo.com
3,3,Charlie Brown,charlie@outlook.com,outlook.com
4,4,David Lee,david@abc.com,abc.com


#### dim_products

In [0]:
%sql
create or replace table datamodel.gold.dim_products
as
with dedup (
    select distinct
        product_id,
        product_name,
        product_category
    from datamodel.silver.silver_tbl
)
select 
    row_number() over (order by product_id) as dim_product_key,
    *
from dedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodel.gold.dim_products;

dim_product_key,product_id,product_name,product_category
1,501,iPhone 14,Electronics
2,502,AirPods Pro,Electronics
3,503,Nike Shoes,Footwear
4,504,Samsung S23,Electronics


#### dim_payments

In [0]:
%sql
create or replace table datamodel.gold.dim_payments
as
with dedup (
    select distinct
        payment_type
    from datamodel.silver.silver_tbl
)
select 
    row_number() over (order by payment_type) as dim_payment_key,
    *
from dedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodel.gold.dim_payments;

dim_payment_key,payment_type
1,Credit Card
2,PayPal


#### dim_countries

In [0]:
%sql
create or replace table datamodel.gold.dim_countries
as
with dedup (
    select distinct
        country
    from datamodel.silver.silver_tbl
)
select 
    row_number() over (order by country) as dim_country_key,
    *
from dedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodel.gold.dim_countries;

dim_country_key,country
1,Canada
2,USA


#### dim_orders

In [0]:
%sql
create or replace table datamodel.gold.dim_orders
as
with dedup (
    select distinct
        order_id,
        order_date,
        customer_id,
        customer_name,
        customer_email,
        customer_email_host,
        product_id,
        product_name,
        product_category,
        payment_type,
        country,
        last_updated,
        processed_date
    from datamodel.silver.silver_tbl
)
select 
    row_number() over (order by order_id) as dim_order_key,
    *
from dedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodel.gold.dim_orders;

dim_order_key,order_id,order_date,customer_id,customer_name,customer_email,customer_email_host,product_id,product_name,product_category,payment_type,country,last_updated,processed_date
1,1001,2024-07-01,1,Alice Johnson,alice@gmail.com,gmail.com,501,iPhone 14,Electronics,Credit Card,USA,2024-07-01,2025-08-11
2,1002,2024-07-01,2,Bob Smith,bob@yahoo.com,yahoo.com,502,AirPods Pro,Electronics,PayPal,USA,2024-07-01,2025-08-11
3,1003,2024-07-01,3,Charlie Brown,charlie@outlook.com,outlook.com,503,Nike Shoes,Footwear,Credit Card,Canada,2024-07-01,2025-08-11
4,1004,2024-07-02,4,David Lee,david@abc.com,abc.com,504,Samsung S23,Electronics,Credit Card,USA,2024-07-02,2025-08-11
5,1005,2024-07-02,1,Alice Johnson,alice@gmail.com,gmail.com,503,Nike Shoes,Footwear,Credit Card,USA,2024-07-02,2025-08-11


#### fact_sales

In [0]:
%sql
create or replace table datamodel.gold.fact_sales
as
select
  od.dim_order_key,
  pd.dim_product_key,
  cu.dim_customer_key,
  pt.dim_payment_key,
  ct.dim_country_key,
  sv.quantity as m_quantity,
  sv.unit_price as m_unit_price
from datamodel.silver.silver_tbl sv
left join datamodel.gold.dim_orders od
  on sv.order_id = od.order_id
left join datamodel.gold.dim_products pd
  on sv.product_id = pd.product_id
left join datamodel.gold.dim_customers cu
  on sv.customer_id = cu.customer_id
left join datamodel.gold.dim_payments pt
  on sv.payment_type = pt.payment_type
left join datamodel.gold.dim_countries ct
  on sv.country = ct.country

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodel.gold.fact_sales;

dim_order_key,dim_product_key,dim_customer_key,dim_payment_key,dim_country_key,m_quantity,m_unit_price
1,1,1,1,2,1,999.99
2,2,2,2,2,2,199.99
3,3,3,1,1,1,129.99
4,4,4,1,2,1,899.99
5,3,1,1,2,2,129.99
